In [6]:
from extract_dataframe import read_json
from extract_dataframe import TweetDfExtractor
from clean_tweets_dataframe import Clean_Tweets

In [ ]:
if __name__ == "__main__":
    _, tweet_list = read_json("data/Economic_Twitter_Data.json")
    tweet = TweetDfExtractor(tweet_list)
    tweet_df = tweet.get_tweet_df()

In [ ]:
tweet_df.shape

In [ ]:
ct = Clean_Tweets(tweet_df)

In [ ]:
df_unwanted_dropped = ct.drop_unwanted_column(tweet_df)
df_unwanted_dropped.shape

In [ ]:
df_duplicate_dropped = ct.drop_duplicate(tweet_df)
df_duplicate_dropped.shape

In [ ]:
df_non_english_removed = ct.remove_non_english_tweets(tweet_df)
df_non_english_removed